In [27]:
from eigSVD import eigsvd

In [28]:
import numpy as np

In [29]:
from scipy.linalg import lu, qr, svd, orth

In [30]:
from numpy.linalg import LinAlgError

In [31]:
def lu_my(A):
    n = A.shape[0]
    U = A.copy()
    L = np.eye(n, dtype=np.double)
    for i in range(n):
        factor = U[i+1:, i] / U[i, i]
        L[i+1:, i] = factor
        U[i+1:] -= factor[:, np.newaxis] * U[i]
    return L, U

In [32]:
def lu_my_with_permute(A):
    n = A.shape[0]
    U = A.copy()
    L = np.eye(n, dtype=np.double)
    P = np.eye(n, dtype=np.double)  # Initialize the permutation matrix P
    for i in range(n):
        # Find the pivot row (row with the maximum absolute value in the current column)
        pivot_row = np.argmax(np.abs(U[i:, i])) + i

        # Swap rows in U and L to perform pivoting
        U[[i, pivot_row]] = U[[pivot_row, i]]
        L[[i, pivot_row]] = L[[pivot_row, i]]
        P[[i, pivot_row]] = P[[pivot_row, i]]

        factor = U[i+1:, i] / U[i, i]
        L[i+1:, i] = factor
        U[i+1:] -= factor[:, np.newaxis] * U[i]


    #return np.dot(P.T,L),
    return l, u

In [33]:
def rSVDBKI(A, k, p, s=5):
    m, n = A.shape

    if s <= 5:
        s = 5

    O = np.random.randn(n, k+s)

    tmp = np.dot(A, O)
    H0, _ = lu(tmp, permute_l=True)
    H = np.array(H0)
    Hi = H0
    for i in range(1, p+1):
        Hi, _ = lu(np.dot(A, np.dot(A.T, Hi)), permute_l=True)
        H = np.hstack((H, Hi))

    Q = orth(H)
    B = np.dot(Q.T, A)

    V, S, U = eigsvd(B.T)

    ind = slice((k+s)*(p+1)-k+1,(k+s)*(p+1))

    U, S, V = np.dot(Q, U)[:, ind], S[ind, ind], V[:, ind]

    return U, S, V